<a href="https://colab.research.google.com/github/LeeSeungwon89/Python_for_Data_Analytics_Science/blob/main/4.%20%EB%8D%B0%EC%9D%B4%ED%84%B0%20%EC%A0%84%EC%B2%98%EB%A6%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **1. 데이터 전처리하기**

데이터를 분석하기 전에 데이터 품질을 제고하는 데이터 전처리(preprocessing) 작업을 선행해야 합니다. 데이터 분석 결과가 왜곡되는 상황을 방지하고 데이터 분석 정확도를 제고하기 위한 필수 작업입니다. 

데이터 분석 강의, 인터넷 정보, 서적, **파이썬 머신러닝 판다스 데이터 분석(오승환, 정보문화사)**을 참고했습니다.

## **1.1. 누락된 데이터**

특정 데이터를 실수로 입력하지 않거나 파일 형식을 변환하면서 데이터가 손상되는 경우가 많습니다. 전처리 작업을 거치지 않으면 정확한 데이터 분석 결과를 얻기 어렵습니다. 

### **1.1.1 누락된 데이터 확인하기**

데이터 분석을 위한 라이브러리 목록입니다. 복사해서 사용합니다.

In [80]:
# 기본적인 라이브러리입니다.
import time
import random
import math

# 데이터 분석을 위한 라이브러리입니다.
import numpy as np
import pandas as pd

# 수학 라이브러리입니다.
import scipy as sp
import statsmodels.api as sm

# 웹 스크레이핑을 위한 라이브러리입니다
import re
import requests
from bs4 import BeautifulSoup
import os
import json

# 시각화 라이브러리입니다.
import matplotlib as mpl
import matplotlib.pylab as plb
import matplotlib.pyplot as plt
import sklearn as sk
import seaborn as sns

# 시각화 자료를 바로 띄워줍니다.
%matplotlib inline

# 그래프에 retina를 지정합니다.
%config InlineBackend.figure_format = 'retina'

# 음수 부호를 깨지지 않도록 합니다.
mpl.rc('axes', unicode_minus=False)

사용할 데이터셋은 `seaborn` 라이브러리의 타이타닉 탑승객 정보 데이터셋입니다.

In [98]:
df = sns.load_dataset('titanic')
df.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


`info()` 메서드만으로도 누락된 데이터를 어느 정도 파악할 수 있습니다. 총 데이터 개수는 891인데 'age', 'embarked', 'deck', 'embark_town' 열은 데이터 수가 891 미만으로 모자랍니다.

참고로 누락된 데이터가 `NaN`으로 잡히지 않는 경우도 있습니다. 특정 기호인 `?`, `!`, `-`, ` ` 등으로 이루어진 값은 `NaN`으로 잡히지 않습니다. 이 경우 `replace()` 메서드와 넘파이의 `np.nan`를 혼용하여 `NaN`으로 바꿀 필요가 있습니다. `*`로 채워진 경우라면 `df.replace('*', np.nan, inplace=True)` 형식으로 바꿀 수 있습니다.

In [83]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   survived     891 non-null    int64   
 1   pclass       891 non-null    int64   
 2   sex          891 non-null    object  
 3   age          714 non-null    float64 
 4   sibsp        891 non-null    int64   
 5   parch        891 non-null    int64   
 6   fare         891 non-null    float64 
 7   embarked     889 non-null    object  
 8   class        891 non-null    category
 9   who          891 non-null    object  
 10  adult_male   891 non-null    bool    
 11  deck         203 non-null    category
 12  embark_town  889 non-null    object  
 13  alive        891 non-null    object  
 14  alone        891 non-null    bool    
dtypes: bool(2), category(2), float64(2), int64(4), object(5)
memory usage: 80.6+ KB


In [82]:
df.describe()

,survived,pclass,age,sibsp,parch,fare
count,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


`isnull()` 메서드를 사용하면 `NaN`을 의미하는 `True`가 1, `False`가 0입니다. 이 값을 열 기준으로 모두 더하는 `sum(axis=0)` 메서드를 활용하면 데이터가 누락된 열 개수를 파악할 수 있습니다. `axis=0`과 `axis=1`은 혼동하기 쉬운 개념입니다. 명확하게 정리하겠습니다.

- `axis=0`: 행(인덱스) 방향으로 작동합니다. 책을 위로 쌓으면서 정리한다고 볼 수 있습니다. 다시 말하면 각 열의 모든 행으로 작동합니다. 작업 결과는 행으로 출력됩니다. 

- `axis=1`: 열(컬럼) 방향으로 작동합니다. 책을 옆으로 정리한다고 볼 수 있습니다. 다시 말하면 각 행의 모든 열로 작동합니다. 작업 결과는 열로 출력됩니다. 

In [84]:
df.head().isnull()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False
1,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False
3,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False


In [85]:
df.isnull().sum(axis=0)

survived         0
pclass           0
sex              0
age            177
sibsp            0
parch            0
fare             0
embarked         2
class            0
who              0
adult_male       0
deck           688
embark_town      2
alive            0
alone            0
dtype: int64

`notnull()` 메서드는 `isnull()` 메서드의 반대 방향으로 작동합니다.

In [86]:
df.head().notnull()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,True,True,True,True,True,True,True,True,True,True,True,False,True,True,True
1,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True
2,True,True,True,True,True,True,True,True,True,True,True,False,True,True,True
3,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True
4,True,True,True,True,True,True,True,True,True,True,True,False,True,True,True


In [87]:
df.notnull().sum(axis=0)

survived       891
pclass         891
sex            891
age            714
sibsp          891
parch          891
fare           891
embarked       889
class          891
who            891
adult_male     891
deck           203
embark_town    889
alive          891
alone          891
dtype: int64

`value_counts()` 메서드는 각 열의 데이터 값마다 개수로 출력합니다. `dropna=False`로 지정하면 `NaN`의 개수도 함께 출력합니다.

In [88]:
df['age'].value_counts(dropna=False)

NaN      177
24.00     30
22.00     27
18.00     26
28.00     25
        ... 
36.50      1
55.50      1
66.00      1
23.50      1
0.42       1
Name: age, Length: 89, dtype: int64

In [89]:
df['deck'].value_counts(dropna=False)

NaN    688
C       59
B       47
D       33
E       32
A       15
F       13
G        4
Name: deck, dtype: int64

반목문으로도 `NaN`을 확인할 수 있습니다. 예외 처리 구문은 [링크](https://wikidocs.net/30)를 참조하시기 바랍니다.

In [90]:
df_nan = df.isnull()

for column in df_nan:
    count_nan = df_nan[column].value_counts()
    try:
        print(f'{column}: {count_nan[True]}')
    # `missing_count`의 데이터에 `True`가 없으면 예외로 처리합니다.
    except:
        print(f'{column}: 0')

survived: 0
pclass: 0
sex: 0
age: 177
sibsp: 0
parch: 0
fare: 0
embarked: 2
class: 0
who: 0
adult_male: 0
deck: 688
embark_town: 2
alive: 0
alone: 0


### **1.1.2. 누락된 데이터 제거하기**

누락된 원소가 있는 행(레코드)을 삭제하거나 열(특성)을 삭제하는 방법을 취할 수 있습니다.

'deck' 열은 누락된 원소가 688개이므로 `dropna()` 메서드를 사용하여 열을 통으로 삭제해야 합니다. `axis=0`으로 지정하여 행을 기준으로 삼아 아래로 카운트하고, `thresh=300`으로 지정하여 `NaN`이 300개 이상 카운트된 열을 삭제하겠습니다.

In [91]:
df_thresh = df.dropna(axis=0, thresh=300)
print(df_thresh.columns)

Index(['survived', 'pclass', 'sex', 'age', 'sibsp', 'parch', 'fare',
       'embarked', 'class', 'who', 'adult_male', 'deck', 'embark_town',
       'alive', 'alone'],
      dtype='object')


'deck' 열만 삭제되었습니다.

'age' 열의 원소는 177개만큼 `NaN`입니다. 데이터를 분석할 때 'age'가 중요한 특성(열, 변수)라면 특성은 남겨두고 `NaN`인 행(레코드)를 제거하는 방법을 취할 수 있습니다. `subset=['age']` 옵션은 'age' 열만 한정한다는 의미이고, `how='any'`(기본값) 옵션은 `NaN`이 하나라도 있으면 삭제한다는 의미입니다. `how='all'` 옵션은 모든 값이 `Nan`이면 삭제한다는 의미입니다.

In [92]:
df_age = df.dropna(subset=['age'], how='any', axis=0)
len(df_age)

714

### **1.1.3. 누락된 데이터 대체하기**

기존 데이터를 최대한 살려서 사용할 수 있도록 누락된 값을 다른 값으로 대체하는 방법을 우선 고려해야 합니다. 무조건 행이나 열을 삭제하는 방법은 데이터를 크게 손상시킵니다.

핵심적으로 쓰이는 메서드는 `fillna()`입니다. `NaN` 값을 다른 값으로 대체하는 메서드입니다. 누락된 값을 대체할 값으로는 **평균값**, **최빈값**, **중간값** 등을 고려할 수 있습니다.

먼저 'age'열의 `NaN` 값을 평균값으로 대체하겠습니다.

In [99]:
age_mean = df['age'].mean(axis=0)
# `inplace=True` 옵션을 지정하여 원본 데이터프레임 객체를 영구적으로 변경합니다.
df['age'].fillna(age_mean, inplace=True)

print(f"NaN: {sum(df['age'].isnull())}개")
print()
print(df['age'].value_counts(dropna=False))

NaN: 0개

29.699118    177
24.000000     30
22.000000     27
18.000000     26
28.000000     25
            ... 
55.500000      1
53.000000      1
20.500000      1
23.500000      1
0.420000       1
Name: age, Length: 89, dtype: int64


`NaN` 값 177개가 평균값인 29.699로 대체되었습니다.

다음은 'embark_town' 열의 `NaN` 값을 최빈값으로 대체하겠습니다. 먼저 `NaN` 값이 존재하는 레코드를 확인하겠습니다. 불리언 인덱싱을 사용합니다.

In [107]:
embark_town_nan = df['embark_town'].isnull()
df[embark_town_nan]

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
61,1,1,female,38.0,0,0,80.0,NaN,First,woman,False,B,NaN,yes,True
829,1,1,female,62.0,0,0,80.0,NaN,First,woman,False,B,NaN,yes,True


`value_counts()` 메서드로 각 값의 개수를 구합니다.

In [111]:
embark_town_value_counts = df['embark_town'].value_counts(dropna=False)
embark_town_value_counts

Southampton    644
Cherbourg      168
Queenstown      77
NaN              2
Name: embark_town, dtype: int64

최빈값은 'Southampton'입니다. `idxmax()` 메서드를 사용하면 최빈값만 출력할 수 있습니다.

In [112]:
embark_town_idxmax = embark_town_value_counts.idxmax()
embark_town_idxmax

'Southampton'

`NaN` 값을 최빈값인 'Southampton'로 대체하고 대체 여부를 확인하겠습니다.

In [150]:
df['embark_town'].fillna(embark_town_idxmax, inplace=True)
df.loc[[61, 829], ['embark_town']]

,embark_town
61,Southampton
829,Southampton


아래 형식으로도 확인할 수도 있지만 가능하면 `loc`과 `iloc`을 사용하는 습관을 들이는 것이 좋습니다.

In [128]:
df['embark_town'][[61, 829]]

61     Southampton
829    Southampton
Name: embark_town, dtype: object

서로 이웃한 값은 유사한 특성을 가졌을 가능성이 높습니다(물론 데이터셋마다 차이가 있습니다). 이 경우 `NaN` 값을 이웃한 값으로 바꾸려면 `fillna()` 메서드에 `method='bfill'` 또는 `method='ffill'` 옵션을 지정합니다. 아래는 이 옵션에 대한 예시입니다. 먼저 새로운 데이터프레임을 생성하겠습니다.

In [145]:
df_ex = pd.DataFrame(np.random.randn(5, 3), columns=['C1', 'C2', 'C3'])

df_ex.iloc[0, 0] = None
df_ex.loc[1, ['C1', 'C3']] = np.nan
df_ex.loc[2, 'C2'] = np.nan
df_ex.loc[3, 'C2'] = np.nan
df_ex.loc[4, 'C3'] = np.nan

df_ex

,C1,C2,C3
0,NaN,0.205386,-0.950875
1,NaN,-1.460394,NaN
2,-1.109804,NaN,-1.835130
3,-1.333353,NaN,0.743006
4,0.192806,-1.711175,NaN


`ffill`을 적용합니다. `NaN` 값에 `NaN` 값의 바로 위 레코드의 값을 채웁니다.

In [146]:
df_ex.fillna(method='ffill')

,C1,C2,C3
0,NaN,0.205386,-0.950875
1,NaN,-1.460394,-0.950875
2,-1.109804,-1.460394,-1.835130
3,-1.333353,-1.460394,0.743006
4,0.192806,-1.711175,0.743006


바로 위 레코드가 없는 `(0, 'C1')`, `(1, 'C1')`는 그대로 `NaN`입니다.

`bfill`을 적용합니다. `NaN` 값에 `NaN` 값의 바로 아래 레코드의 값을 채웁니다.

In [147]:
df_ex.fillna(method='bfill')

,C1,C2,C3
0,-1.109804,0.205386,-0.950875
1,-1.109804,-1.460394,-1.835130
2,-1.109804,-1.711175,-1.835130
3,-1.333353,-1.711175,0.743006
4,0.192806,-1.711175,NaN


## **1.2. 중복된 데이터**

### **1.2.1. 중복된 데이터 확인하기**

### **1.2.2. 중복된 데이터 제거하기**

## **1.3. 데이터 표준화**

### **1.3.1. 단위 환산하기**

### **1.3.2. 자료형 변환하기**

## **1.4. 범주형 데이터**

### **1.4.1. 구간 분할하기**

### **1.4.2. 더미 변수 사용하기**

## **1.5. 데이터 정규화**

## **1.6. 시계열 데이터**

### **1.6.1. 시계열 객체로 변환하기**

### **1.6.2. 시계열 데이터 생성하기**

### **1.6.3. 시계열 데이터 활용하기**